# Missing data imputation with Fedbiomed using MIWAE, using ADNI (non-iid distributed)

In this notebook we show:
* how to obtain mean and std in a federated manner, to perform afterwards local dataset standardization with respect to the global dataset
* how to impute missing not at random (MAR) data in a federated setting using MIWAE (https://arxiv.org/abs/2006.12871). 

We will compare results of federated training using FedAvg, FedProx (with both local standardization and federated standardization), with local results.

We are going to use data extracted from ADNI. In particular, we consider 311 participants extracted from the ADNI dataset, among cognitively normal (NL) (104 subjects) and patients diagnosed with AD (207 subjects). All participants are associated with multiple data views: cognitive scores including MMSE, CDR-SB, ADAS-Cog-11 and RAVLT (CLINIC), Magnetic resonance imaging (MRI), Fluorodeoxyglucose-PET (FDG) and AV45-Amyloid PET (AV45) images. We are going to generate a test dataset containing the 20% of the samples (using `train_test_split` from `sklearn.model_selection`), while the training dataset will be further distributed across 2 clients, one containing exclusively patients affectd with Alzheimer Disease (171 sample), and the other one containing only healthy patients (77 sample). From each local dataset we will remove randomly 30% of the observations, while we remove 50 of observations randomly from the test dataset.

In [ ]:
%load_ext autoreload
%autoreload 2

## Prepare the data (skip this part if done offline)

For this experiment we will use the data extracted from ADNI.

In [ ]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

cwd = os.getcwd()
data_file = os.path.join(cwd, "data/ADNI/data_irene_dx_2g_corrected.csv")
raw_df = pd.read_csv(data_file, sep=",",index_col="RID")
target = raw_df["DX"]
data = deepcopy(raw_df)
del data["DX"]
print(data.shape)

In [ ]:
from sklearn.model_selection import train_test_split

#train test split
data_train, data_test, labels_train, labels_test = train_test_split(data, target, test_size=0.20, random_state=42)

# split train across datasets in a non-iid manner: 
#client 1 will only contain ADNI subjects, client 2 will only contain controls
client_1 = data_train.loc[labels_train == 'AD']
client_2 = data_train.loc[labels_train == 'NL']

#Clients_data contains original full data of each client 
Clients_data=[client_1, client_2]

In [ ]:
# from each dataset we will remove randomly 30% of data
np.random.seed(1234)

# 30% of missing data for client 1, 30% for client 2
perc_miss_list = [0.3,0.3] 

#Clients_data contains data of each client with missing entries wrt to perc_miss_list
Clients_missing = []
for perc,c in enumerate(Clients_data):
    perc_miss=perc_miss_list[perc]
    n = c.shape[0] # number of observations
    p = c.shape[1] # number of features
    xmiss = np.copy(c)
    xmiss_flat = xmiss.flatten()
    miss_pattern = np.random.choice(n*p, np.floor(n*p*perc_miss).astype(np.int_),\
                                    replace=False)
    xmiss_flat[miss_pattern] = np.nan 
    xmiss = xmiss_flat.reshape([n,p]) # in xmiss, the missing values are represented by nans
    mask = np.isfinite(xmiss) # binary mask that indicates which values are missing
    Clients_missing.append(xmiss)

Finally we save the datasets, which will be distributed across three nodes:

In [ ]:
import os 
os.makedirs('data/clients_data', exist_ok=True) 
for i in range(len(Clients_missing)):
    pd.DataFrame(Clients_missing[i]).to_csv('data/clients_data/client_'+str(i+1)+'.csv',index=False)

## Recover full dataset and test dataset for testing phase

In [ ]:
import pandas as pd
import numpy as np
import os 

cwd = os.getcwd()

N_cl = 2
Split_type='notIID'

Clients_data=[]
Clients_missing=[]
for i in range(N_cl):
    data_full_file = os.path.join(cwd, "data/clients_data/client_full_"+Split_type+"_"+str(i+1)+".csv")
    data_full = pd.read_csv(data_full_file, sep=",",index_col=False)
    Clients_data.append(data_full)
    data_file = os.path.join(cwd, "data/clients_data/client_"+Split_type+"_"+str(i+1)+".csv")
    data = pd.read_csv(data_file, sep=",",index_col=False)
    Clients_missing.append(data)

test_file = os.path.join(cwd, "data/test_data/test_full_"+Split_type+".csv")
data_test = pd.read_csv(test_file, sep=",",index_col=False)
test_missing_file = os.path.join(cwd, "data/test_data/test_"+Split_type+".csv")
data_test_missing = pd.read_csv(test_missing_file, sep=",",index_col=False)

In [ ]:
import csv, os

os.makedirs('results', exist_ok=True) 
if not os.path.exists('results/output_notiid.csv'):
    output = open("results/output_notiid.csv", "w")
    writer = csv.DictWriter(output, 
                            fieldnames=['Split_type', 'Test_data', 'model', 
                                        'N_train_centers', 'Size', 'N_rounds', 'N_epochs',
                                        'std_training', 'std_testing', 'MSE'])
    writer.writeheader()
    output.close()

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the nodes up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 1 (csv) to add client_1 dataset to the first node
  * Provide the correct tag by entering:  `adni`
  * Pick the folder where client_1 dataset has been saved
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node start`. Wait until you get `Starting task manager`. it means you are online.
4. Following the same procedure, you can create additional nodes for clients 2 and 3.

Check available clients:

In [ ]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)
xx = req.list()
dataset_size = [xx[i][0]['shape'][1] for i in xx]
assert min(dataset_size)==max(dataset_size)
data_size = dataset_size[0]

## Recover global mean and std

In [ ]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
from copy import deepcopy

from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from fedbiomed.common.constants import ProcessTypes

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class FedMeanStdTrainingPlan(TorchTrainingPlan):
    
    def init_dependencies(self):
        deps = ["import pandas as pd",
               "import numpy as np",
               "from copy import deepcopy"]
        return deps
        
    def init_model(self,model_args):
        
        model = self.MeanStd(model_args)
        
        return model
    
    class MeanStd(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.n_features=model_args['n_features']
            
            self.mean = nn.Parameter(torch.zeros(self.n_features,dtype=torch.float64),requires_grad=False)
            self.std = nn.Parameter(torch.zeros(self.n_features,dtype=torch.float64),requires_grad=False)
            self.size = nn.Parameter(torch.zeros(self.n_features,dtype=torch.float64),requires_grad=False)
            self.fake = nn.Parameter(torch.randn(1),requires_grad=True)

        def forward(self, data):
            data_np = data.numpy()
        
            ### Implementing with np.nanmean, np.nanstd
            
            self.size += torch.Tensor([data_np[:,dim].size - np.count_nonzero(np.isnan(data_np[:,dim]))\
                                       for dim in range(self.n_features)])
            self.mean += torch.from_numpy(np.nanmean(data_np,0))
            self.std += torch.from_numpy(np.nanstd(data_np,0))
            
            # ### Implementing with torch.mean, torch.std
        
            # size_loc = torch.zeros(self.n_features)
            # mean_loc = torch.zeros(self.n_features)
            # std_loc = torch.zeros(self.n_features)
            # for dim in range(self.n_features):
            #     data_i = deepcopy(data[:,dim][mask[:,dim].bool()])
            #     size_loc[dim] = data_i.shape[0]
            #     mean_loc[dim] = torch.mean(data_i, dim=0)
            #     std_loc[dim] = torch.std(data_i, unbiased=False, dim=0)
            # self.size += size_loc
            # self.mean += mean_loc
            # self.std += std_loc
            
            return self.fake
    
        
    def training_data(self):
        
        df = pd.read_csv(self.dataset_path, sep=',', index_col=False)
        
        ### NOTE: batch_size should be == dataset size ###
        batch_size = df.shape[0]
        x_train = df.values
        x_mask = np.isfinite(x_train)
        xhat_0 = np.copy(x_train)
        ### NOTE: we keep nan when data is missing
        #xhat_0[np.isnan(x_train)] = 0
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        
        data_manager = DataManager(dataset=xhat_0 , target=x_mask , **train_kwargs)
        
        return data_manager
    
    def training_step(self, data, mask):
        
        return self.model().forward(data)

In [ ]:
# NOTE: we need to perform only 1 round of 1 epoch

model_args = {'n_features':data_size}

training_args = {
    'batch_size': 48, 
    'optimizer_args': {
        'lr': 0
    }, 
    'log_interval' : 1,
    'epochs': 1, 
    'dry_run': False,  
    #'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

tags =  ['adni']

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedstandard import FedStandard

fed_mean_std = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=FedMeanStdTrainingPlan,
                 training_args=training_args,
                 round_limit=1,
                 aggregator=FedStandard(),
                 node_selection_strategy=None)

In [ ]:
fed_mean_std.run()

In [ ]:
fed_mean = fed_mean_std.aggregated_params()[0]['params']['fed_mean']
fed_std = fed_mean_std.aggregated_params()[0]['params']['fed_std']

## Define an experiment model and parameters

Declare a torch.nn MIWAETrainingPlan class to send for training on the node

Note: we include a function, ``standardize_data``, which allow to standardize data either with respect to a mean and std provided by the user, or locally, considering only local data for each client.

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import numpy as np
import torch.distributions as td
import pandas as pd

from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from fedbiomed.common.constants import ProcessTypes

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MIWAETrainingPlan(TorchTrainingPlan):
    
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms",
               "import torch.distributions as td",
               "import pandas as pd",
               "import numpy as np"]
        return deps
        
    def init_model(self,model_args):
        
        if 'standardization' in model_args:
            self.standardization = True
            if (('fed_mean' in model_args['standardization']) and ('fed_std' in model_args['standardization'])):
                self.fed_mean = np.array(model_args['standardization']['fed_mean'])
                self.fed_std = np.array(model_args['standardization']['fed_std'])
            else:
                self.fed_mean = None
                self.fed_std = None
                
        self.n_features=model_args['n_features']
        self.n_latent=model_args['n_latent']
        self.n_hidden=model_args['n_hidden']
        self.n_samples=model_args['n_samples']
        
        model = self.MIWAE(model_args)
        
        return model
    
    class MIWAE(nn.Module):
        def __init__(self, model_args):
            super().__init__()

            n_features=model_args['n_features']
            n_latent=model_args['n_latent']
            n_hidden=model_args['n_hidden']
            n_samples=model_args['n_samples']

            # the encoder will output both the mean and the diagonal covariance
            self.encoder=nn.Sequential(
                            torch.nn.Linear(n_features, n_hidden),
                            torch.nn.ReLU(),
                            torch.nn.Linear(n_hidden, n_hidden),
                            torch.nn.ReLU(),
                            torch.nn.Linear(n_hidden, 2*n_latent),  
                            )
            # the decoder will output both the mean, the scale, 
            # and the number of degrees of freedoms (hence the 3*p)
            self.decoder = nn.Sequential(
                            torch.nn.Linear(n_latent, n_hidden),
                            torch.nn.ReLU(),
                            torch.nn.Linear(n_hidden, n_hidden),
                            torch.nn.ReLU(),
                            torch.nn.Linear(n_hidden, 3*n_features),  
                            )

            self.encoder.apply(self.weights_init)
            self.decoder.apply(self.weights_init)
    
        def weights_init(self,layer):
            if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)
    
    def init_optimizer(self,optimizer_args):
        
        optimizer = torch.optim.Adam(list(self.model().encoder.parameters()) \
                                    + list(self.model().decoder.parameters()),lr = optimizer_args['lr'])
        
        return optimizer
        
        
    def miwae_loss(self,iota_x,mask):
        # prior
        self.p_z = td.Independent(td.Normal(loc=torch.zeros(self.n_latent).to(self._device)\
                                       ,scale=torch.ones(self.n_latent).to(self._device)),1)
        
        batch_size = iota_x.shape[0]
        out_encoder = self.model().encoder(iota_x)
        
        q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :self.n_latent],\
                                                scale=torch.nn.Softplus()\
                                                (out_encoder[..., self.n_latent:\
                                                             (2*self.n_latent)])),1)

        zgivenx = q_zgivenxobs.rsample([self.n_samples])
        zgivenx_flat = zgivenx.reshape([self.n_samples*batch_size,self.n_latent])

        out_decoder = self.model().decoder(zgivenx_flat)
        all_means_obs_model = out_decoder[..., :self.n_features]
        all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., self.n_features:\
                                                               (2*self.n_features)]) + 0.001
        all_degfreedom_obs_model = torch.nn.Softplus()\
        (out_decoder[..., (2*self.n_features):(3*self.n_features)]) + 3

        data_flat = torch.Tensor.repeat(iota_x,[self.n_samples,1]).reshape([-1,1])
        tiledmask = torch.Tensor.repeat(mask,[self.n_samples,1])

        all_log_pxgivenz_flat = torch.distributions.StudentT\
        (loc=all_means_obs_model.reshape([-1,1]),\
         scale=all_scales_obs_model.reshape([-1,1]),\
         df=all_degfreedom_obs_model.reshape([-1,1])).log_prob(data_flat)
        all_log_pxgivenz = all_log_pxgivenz_flat.reshape([self.n_samples*batch_size,self.n_features])

        logpxobsgivenz = torch.sum(all_log_pxgivenz*tiledmask,1).reshape([self.n_samples,batch_size])
        logpz = self.p_z.log_prob(zgivenx)
        logq = q_zgivenxobs.log_prob(zgivenx)

        neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq,0))

        return neg_bound

    def training_data(self,  batch_size = 48):
        
        df = pd.read_csv(self.dataset_path, sep=',', index_col=False)
        x_train = df.values
        x_mask = np.isfinite(x_train)
        # xhat_0: missing values are replaced by zeros. 
        #This x_hat0 is what will be fed to our encoder.
        xhat_0 = np.copy(x_train)
        
        # Data standardization
        if self.standardization:
            xhat_0 = self.standardize_data(xhat_0)
            
        xhat_0[np.isnan(x_train)] = 0
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        
        data_manager = DataManager(dataset=xhat_0 , target=x_mask , **train_kwargs)
        
        return data_manager
    
    def standardize_data(self,data):
        data_norm = np.copy(data)
        if ((self.fed_mean is not None) and (self.fed_std is not None)):
            print('FEDERATED STANDARDIZATION')
            data_norm = (data_norm - self.fed_mean)/self.fed_std
        else:
            print('LOCAL STANDARDIZATION')
            data_norm = (data_norm - np.nanmean(data_norm,0))/np.nanstd(data_norm,0)
        return data_norm
    
    def training_step(self, data, mask):
        self.model().encoder.zero_grad()
        self.model().decoder.zero_grad()
        loss = self.miwae_loss(iota_x = data,mask = mask)
        return loss

This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. 
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.
* data `tags` to search nodes for training.
* total number of `rounds`.
If FedProx optimisation is requested, `fedprox_mu` parameter must be defined here. It also must be a float between XX and YY.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [ ]:
h = 128 # number of hidden units in (same for all MLPs)
d = 5 # dimension of the latent space, we choose d=1 for visualisation purposes
K = 100 # number of IS during training

n_epochs=10
batch_size = 32

model_args = {'n_features':data_size, 'n_latent':d,'n_hidden':h,'n_samples':K, 'use_gpu': True,
             'standardization':{'fed_mean':fed_mean.tolist(),'fed_std':fed_std.tolist()}}

training_args = {
    'batch_size': batch_size, 
    'optimizer_args':
    {'lr': 1e-3}, 
    'log_interval' : 1,
    'epochs': n_epochs, 
    'dry_run': False,  
    #'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

tags =  ['adni']
rounds = 150

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MIWAETrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run()

## Run the experiment with FedProx

We repeat the federated training but using FedProx as aggregation scheme (starting from the second iteration).

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# During the first round we will simply use FedAvg 
# with standard optimization scheme: the FedProx penalization
# term will be introduced exclusively from the second round.
# training_args.update(fedprox_mu = 0.)
if 'fedprox_mu' in training_args:
    del training_args['fedprox_mu'] 

exp_fedprox = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MIWAETrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

In [ ]:
exp_fedprox.run_once()

In [ ]:
# Starting from the second round, FedProx is used with mu=0.1
# We first update the training args
training_args.update(fedprox_mu = 0.1)

# Then update training args in the experiment
exp_fedprox.set_training_args(training_args)
exp_fedprox.run()

## Run the experiment with FedProx and performing the standardization locally

And finally we propose to use FedProx aagain, but this time each client standardize his dataset using local mean and std:

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

if 'fedprox_mu' in training_args:
    del training_args['fedprox_mu'] 

model_args.update(standardization = {})

exp_fedprox_std_local = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MIWAETrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

In [ ]:
exp_fedprox_std_local.run_once()

In [ ]:
training_args.update(fedprox_mu = 0.1)

exp_fedprox_std_local.set_training_args(training_args)
exp_fedprox_std_local.run()

# Test and comparison to local training

We define the imputation function:

In [ ]:
L = 1000

def miwae_impute(encoder,decoder,iota_x,mask,d,L):
    
    p_z = td.Independent(td.Normal(loc=torch.zeros(d),scale=torch.ones(d)),1)
    
    batch_size = iota_x.shape[0]
    out_encoder = encoder(iota_x)
    q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d],scale=torch.nn.Softplus()(out_encoder[..., d:(2*d)])),1)

    zgivenx = q_zgivenxobs.rsample([L])
    zgivenx_flat = zgivenx.reshape([L*batch_size,d])

    out_decoder = decoder(zgivenx_flat)
    all_means_obs_model = out_decoder[..., :p]
    all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., p:(2*p)]) + 0.001
    all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*p):(3*p)]) + 3

    data_flat = torch.Tensor.repeat(iota_x,[L,1]).reshape([-1,1])
    tiledmask = torch.Tensor.repeat(mask,[L,1])

    all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1,1]),scale=all_scales_obs_model.reshape([-1,1]),df=all_degfreedom_obs_model.reshape([-1,1])).log_prob(data_flat)
    all_log_pxgivenz = all_log_pxgivenz_flat.reshape([L*batch_size,p])

    logpxobsgivenz = torch.sum(all_log_pxgivenz*tiledmask,1).reshape([L,batch_size])
    logpz = p_z.log_prob(zgivenx)
    logq = q_zgivenxobs.log_prob(zgivenx)

    xgivenz = td.Independent(td.StudentT(loc=all_means_obs_model, scale=all_scales_obs_model, df=all_degfreedom_obs_model),1)

    imp_weights = torch.nn.functional.softmax(logpxobsgivenz + logpz - logq,0) # these are w_1,....,w_L for all observations in the batch
    xms = xgivenz.mean.reshape([L,batch_size,p])  # that's the only line that changed!
    xm=torch.einsum('ki,kij->ij', imp_weights, xms) 

    return xm

As well as the MSE function:

In [ ]:
def mse(xhat,xtrue,mask): # MSE function for imputations
    xhat = np.array(xhat)
    xtrue = np.array(xtrue)
    return np.mean(np.power(xhat-xtrue,2)[~mask])

## 1. Testing on an external dataset

First of all we are going to test the performance of the final federated model to impute missing data on a test dataset. To this extent we are going to remove randomly 50% of samples from the test dataset, `data_test`, defined at the beginning of this notebook.

In [ ]:
n = data_test_missing.shape[0] # number of observations
p = data_test_missing.shape[1] # number of features

xmiss_test = np.copy(data_test_missing)
mask_test = np.isfinite(xmiss_test) # binary mask that indicates which values are missing

# Evaluate local mean and std of test dataset
mean_test = np.nanmean(xmiss_test,0)
std_test = np.nanstd(xmiss_test,0)

# Evaluate global mean and std including test dataset
#N_cl_test = [fed_mean_std.aggregated_params()[0]['params']['N_tot'],
#           torch.Tensor([xmiss_test[:,dim].size - np.count_nonzero(np.isnan(xmiss_test[:,dim]))\
#                                   for dim in range(p)])]
#mean_cl_test = [fed_mean,torch.from_numpy(mean_test)]
#std_cl_test = [fed_std,torch.from_numpy(std_test)]

#cl_test = 2 # 1 global training dataset + 1 test dataset
#N_tot_cl_test = sum([N_cl_test[c] for c in range(cl_test)])
#Mean_cl_test = sum([N_cl_test[i]*mean_cl_test[i]/N_tot_cl_test for i in range(cl_test)])
#Std_cl_test = torch.sqrt(sum([((N_cl_test[i]-1)*(std_cl_test[i]**2)+N_cl_test[i]*(mean_cl_test[i]**2))/(N_tot_cl_test-cl_test)\
#                              for i in range(cl_test)])-(N_tot_cl_test/(N_tot_cl_test-cl_test))*(Mean_cl_test**2))

# standardization with respect to the fed dataset
xmiss_test_global_std = np.copy(data_test_missing)
xmiss_test_global_std = (xmiss_test_global_std - fed_mean.numpy())/fed_std.numpy()
xhat_0_test_global_std = np.copy(xmiss_test_global_std)
xhat_0_test_global_std[np.isnan(xmiss_test_global_std)] = 0
xhat_test_global_std = np.copy(xhat_0_test_global_std) # This will be out imputed data matrix
xfull_test_global_std = np.copy(data_test)
xfull_test_global_std = (xfull_test_global_std - fed_mean.numpy())/fed_std.numpy()

# local standardization
xmiss_test_local_std = np.copy(data_test_missing)
xmiss_test_local_std = (xmiss_test_local_std - mean_test)/std_test
xhat_0_test_local_std = np.copy(xmiss_test_local_std)
xhat_0_test_local_std[np.isnan(xmiss_test_local_std)] = 0
xhat_test_local_std = np.copy(xhat_0_test_local_std) # This will be out imputed data matrix
xfull_test_local_std = np.copy(data_test)
xfull_test_local_std = (xfull_test_local_std - mean_test)/std_test

We instantiate the model using last updated federated parameters:

In [ ]:
# extract federated model into PyTorch framework
model = exp.model_instance()
model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

encoder = model.encoder
decoder = model.decoder

And we finally do the imputation and evaluate the corresponding imputation error through MSE for each federated model:

In [ ]:
xhat_test = np.copy(xhat_test_global_std)
xhat_0_test = np.copy(xhat_0_test_global_std)
xfull_test = np.copy(xfull_test_global_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder,decoder = decoder,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_test_data_global_std = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of fed model on testing data %g' %err_test_data_global_std)
print('-----')

xhat_test = np.copy(xhat_test_local_std)
xhat_0_test = np.copy(xhat_0_test_local_std)
xfull_test = np.copy(xfull_test_local_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder,decoder = decoder,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_test_data_local_std = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of fed model on testing data (with local standardization in testing data) %g' %err_test_data_local_std)
print('-----')

And save the results in the ouptut file.

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'FedAvg', 
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'global', 'MSE': float(err_test_data_global_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'FedAvg',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'local', 'MSE': float(err_test_data_local_std)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

Same for fedprox

In [ ]:
# extract federated model with fedprox
model_fedprox = exp_fedprox.model_instance()
model_fedprox.load_state_dict(exp_fedprox.aggregated_params()[rounds - 1]['params'])

encoder_fedprox = model_fedprox.encoder
decoder_fedprox = model_fedprox.decoder

xhat_test = np.copy(xhat_test_global_std)
xhat_0_test = np.copy(xhat_0_test_global_std)
xfull_test = np.copy(xfull_test_global_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_test_data_fedprox_global_std = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of fed model (with fedprox) on testing data  %g' %err_test_data_fedprox_global_std)
print('-----')

xhat_test = np.copy(xhat_test_local_std)
xhat_0_test = np.copy(xhat_0_test_local_std)
xfull_test = np.copy(xfull_test_local_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_test_data_fedprox_local_std = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of fed model (with fedprox) on testing data (with local standardization in testing data)  %g' %err_test_data_fedprox_local_std)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'global', 'MSE': float(err_test_data_fedprox_global_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'local', 'MSE': float(err_test_data_fedprox_local_std)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

## 2. Testing on the client's datasets

We are now going to use the final federated model to impute missing data of client 1, which have been used for training:

In [ ]:
# We first recover data (full and with missing entries) from client 1
n = Clients_data[0].shape[0] # number of observations
p = Clients_data[0].shape[1] # number of features

xmiss_cl1 = np.copy(Clients_missing[0])
mask_cl1 = np.isfinite(xmiss_cl1) # binary mask that indicates which values are missing
mean_cl1 = np.nanmean(xmiss_cl1,0)
std_cl1 = np.nanstd(xmiss_cl1,0)

# standardization with respect to the whole dataset (training+test)
xmiss_cl1_global_std = np.copy(Clients_missing[0])
xmiss_cl1_global_std = (xmiss_cl1_global_std - fed_mean.numpy())/fed_std.numpy()
xhat_0_cl1_global_std = np.copy(xmiss_cl1_global_std)
xhat_0_cl1_global_std[np.isnan(xmiss_cl1_global_std)] = 0
xhat_cl1_global_std = np.copy(xhat_0_cl1_global_std) # This will be out imputed data matrix
xfull_cl1_global_std = np.copy(Clients_data[0])
xfull_cl1_global_std = (xfull_cl1_global_std - fed_mean.numpy())/fed_std.numpy()

# local standardization
xmiss_cl1_local_std = np.copy(Clients_missing[0])
xmiss_cl1_local_std = (xmiss_cl1_local_std - mean_cl1)/std_cl1
xhat_0_cl1_local_std = np.copy(xmiss_cl1_local_std)
xhat_0_cl1_local_std[np.isnan(xmiss_cl1_local_std)] = 0
xhat_cl1_local_std = np.copy(xhat_0_cl1_local_std) # This will be out imputed data matrix
xfull_cl1_local_std = np.copy(Clients_data[0])
xfull_cl1_local_std = (xfull_cl1_local_std - mean_cl1)/std_cl1

In [ ]:
### Now we do the imputation

xhat_cl1 = np.copy(xhat_cl1_global_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_global_std)
xfull_cl1 = np.copy(xfull_cl1_global_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder,decoder = decoder, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_global_std = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model on data from client 1  %g' %err_cl1_data_global_std)
print('-----')

xhat_cl1 = np.copy(xhat_cl1_local_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_local_std)
xfull_cl1 = np.copy(xfull_cl1_local_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder,decoder = decoder, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_local_std = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model on data from client 1 (with local standardization in client 1 data) %g' %err_cl1_data_local_std)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'FedAvg',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'global', 'MSE': float(err_cl1_data_global_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'FedAvg',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'local', 'MSE': float(err_cl1_data_local_std)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

In [ ]:
xhat_cl1 = np.copy(xhat_cl1_global_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_global_std)
xfull_cl1 = np.copy(xfull_cl1_global_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_fedprox_global_std = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model (with fedprox) on data from client 1  %g' %err_cl1_data_fedprox_global_std)
print('-----')

xhat_cl1 = np.copy(xhat_cl1_local_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_local_std)
xfull_cl1 = np.copy(xfull_cl1_local_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_fedprox_local_std = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model (with fedprox) on data from client 1 (with local standardization in client 1 data) %g' %err_cl1_data_fedprox_local_std)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'global', 'MSE': float(err_cl1_data_fedprox_global_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'local', 'MSE': float(err_cl1_data_fedprox_local_std)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

And on client 2

In [ ]:
# We first recover data (full and with missing entries) from client 1
n = Clients_data[1].shape[0] # number of observations
p = Clients_data[1].shape[1] # number of features

xmiss_cl2 = np.copy(Clients_missing[1])
mask_cl2 = np.isfinite(xmiss_cl2) # binary mask that indicates which values are missing
mean_cl2 = np.nanmean(xmiss_cl2,0)
std_cl2 = np.nanstd(xmiss_cl2,0)

# standardization with respect to the whole dataset (training+test)
xmiss_cl2_global_std = np.copy(Clients_missing[1])
xmiss_cl2_global_std = (xmiss_cl2_global_std - fed_mean.numpy())/fed_std.numpy()
xhat_0_cl2_global_std = np.copy(xmiss_cl2_global_std)
xhat_0_cl2_global_std[np.isnan(xmiss_cl2_global_std)] = 0
xhat_cl2_global_std = np.copy(xhat_0_cl2_global_std) # This will be out imputed data matrix
xfull_cl2_global_std = np.copy(Clients_data[1])
xfull_cl2_global_std = (xfull_cl2_global_std - fed_mean.numpy())/fed_std.numpy()

# local standardization
xmiss_cl2_local_std = np.copy(Clients_missing[1])
xmiss_cl2_local_std = (xmiss_cl2_local_std - mean_cl2)/std_cl2
xhat_0_cl2_local_std = np.copy(xmiss_cl2_local_std)
xhat_0_cl2_local_std[np.isnan(xmiss_cl2_local_std)] = 0
xhat_cl2_local_std = np.copy(xhat_0_cl2_local_std) # This will be out imputed data matrix
xfull_cl2_local_std = np.copy(Clients_data[1])
xfull_cl2_local_std = (xfull_cl2_local_std - mean_cl2)/std_cl2

In [ ]:
xhat_cl2 = np.copy(xhat_cl2_global_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_global_std)
xfull_cl2 = np.copy(xfull_cl2_global_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder,decoder = decoder, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_cl2_data_global_std = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of fed model on data from client 2  %g' %err_cl2_data_global_std)
print('-----')

xhat_cl2 = np.copy(xhat_cl2_local_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_local_std)
xfull_cl2 = np.copy(xfull_cl2_local_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder,decoder = decoder, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_cl2_data_local_std = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of fed model on data from client 2 (with local standardization in client 2 data) %g' %err_cl2_data_local_std)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'FedAvg',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'global', 'MSE': float(err_cl2_data_global_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'FedAvg',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'local', 'MSE': float(err_cl2_data_local_std)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

In [ ]:
xhat_cl2 = np.copy(xhat_cl2_global_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_global_std)
xfull_cl2 = np.copy(xfull_cl2_global_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_cl2_data_fedprox_global_std = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of fed model (with fedprox) on data from client 2  %g' %err_cl2_data_fedprox_global_std)
print('-----')

xhat_cl2 = np.copy(xhat_cl2_local_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_local_std)
xfull_cl2 = np.copy(xfull_cl2_local_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_cl2_data_fedprox_local_std = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of fed model (with fedprox) on data from client 2 (with local standardization in client 2 data) %g' %err_cl2_data_fedprox_local_std)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'global', 'MSE': float(err_cl2_data_fedprox_global_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Fed', 'std_testing': 'local', 'MSE': float(err_cl2_data_fedprox_local_std)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

## 3. Testing of FedProx model with local standardization

We are going to test the federated model with FedProx, where data standardization is performed locally. In order to be as much coherent as possible, each time the standardization will be realized locally as well in the testing phase.

In [ ]:
# We recover the model
model_fedprox_std_local = exp_fedprox_std_local.model_instance()
model_fedprox_std_local.load_state_dict(exp_fedprox_std_local.aggregated_params()[rounds - 1]['params'])

encoder_fedprox_std_local = model_fedprox_std_local.encoder
decoder_fedprox_std_local = model_fedprox_std_local.decoder

# We do the imputation on the test data, standardized locally
# (since information on the global mean/std are supposed not being available)
n = data_test.shape[0] # number of observations
p = data_test.shape[1] # number of features

xhat_test = np.copy(xhat_test_local_std)
xhat_0_test = np.copy(xhat_0_test_local_std)
xfull_test = np.copy(xfull_test_local_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_fedprox_std_local,decoder = decoder_fedprox_std_local,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_test_data_fedprox_std_local = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of fed model (with fedprox and local standardization) on testing data  %g' %err_test_data_fedprox_std_local)
print('-----')

# Same for the dataset from client 1. 
# In this case the dataset standardization is done with respect to his own data.
n = Clients_data[0].shape[0] # number of observations
p = Clients_data[0].shape[1] # number of features

xhat_cl1 = np.copy(xhat_cl1_local_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_local_std)
xfull_cl1 = np.copy(xfull_cl1_local_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_fedprox_std_local,decoder = decoder_fedprox_std_local, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_fedprox_std_local = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model (with fedprox and local standardization) on data from client 1  %g' %err_cl1_data_fedprox_std_local)
print('-----')

# And the dataset from client 2. 
# In this case the dataset standardization is done with respect to his own data.
n = Clients_data[1].shape[0] # number of observations
p = Clients_data[1].shape[1] # number of features

xhat_cl2 = np.copy(xhat_cl2_local_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_local_std)
xfull_cl2 = np.copy(xfull_cl2_local_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder_fedprox_std_local,decoder = decoder_fedprox_std_local, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_cl2_data_fedprox_std_local = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of fed model (with fedprox and local standardization) on data from client 2  %g' %err_cl2_data_fedprox_std_local)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_test_data_fedprox_std_local)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_cl1_data_fedprox_std_local)}
dict_out_3={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'FedProx',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[i]) for i in range(N_cl)], 
            'N_rounds': rounds, 'N_epochs': n_epochs,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_cl2_data_fedprox_std_local)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    dictwriter_object.writerow(dict_out_3)
    output_file.close()

## 4. Local training on client 1,  and testing

Finally, we test the performance of the same model trained locally and tested on the dataset from client 1. We will use a total of `epochs`x`rounds` local epochs.

In [ ]:
p_z = td.Independent(td.Normal(loc=torch.zeros(d),scale=torch.ones(d)),1)
def miwae_loss(encoder, decoder, iota_x, mask, d, p, K, batch_size):
    
    batch_size = iota_x.shape[0]
    out_encoder = encoder(iota_x)

    q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d],scale=torch.nn.Softplus()(out_encoder[..., d:(2*d)])),1)

    zgivenx = q_zgivenxobs.rsample([K])
    zgivenx_flat = zgivenx.reshape([K*batch_size,d])

    out_decoder = decoder(zgivenx_flat)
    all_means_obs_model = out_decoder[..., :p]
    all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., p:(2*p)]) + 0.001
    all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*p):(3*p)]) + 3

    data_flat = torch.Tensor.repeat(iota_x,[K,1]).reshape([-1,1])
    tiledmask = torch.Tensor.repeat(mask,[K,1])

    all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1,1]),scale=all_scales_obs_model.reshape([-1,1]),df=all_degfreedom_obs_model.reshape([-1,1])).log_prob(data_flat)
    all_log_pxgivenz = all_log_pxgivenz_flat.reshape([K*batch_size,p])

    logpxobsgivenz = torch.sum(all_log_pxgivenz*tiledmask,1).reshape([K,batch_size])
    logpz = p_z.log_prob(zgivenx)
    logq = q_zgivenxobs.log_prob(zgivenx)

    neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq,0))

    return neg_bound

We perform the local training:

In [ ]:
# Recall all hyperparameters

n_epochs_local = n_epochs*rounds

bs = training_args.get('batch_size')
lr = training_args.get('lr')

h = model_args.get('n_hidden') 
d = model_args.get('n_latent') 
K = model_args.get('n_samples') 

# Data

n = Clients_data[0].shape[0] # number of observations
p = Clients_data[0].shape[1] # number of features

xhat_cl1 = np.copy(xhat_cl1_local_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_local_std)
xfull_cl1 = np.copy(xfull_cl1_local_std)

encoder_cl1 = nn.Sequential(
    torch.nn.Linear(p, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 2*d),  # the encoder will output both the mean and the diagonal covariance
)

decoder_cl1 = nn.Sequential(
    torch.nn.Linear(d, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 3*p),  # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
)

optimizer_cl1 = torch.optim.Adam(list(encoder_cl1.parameters()) + list(decoder_cl1.parameters()),lr=1e-3)

def weights_init(layer):
    if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)
        
encoder_cl1.apply(weights_init)
decoder_cl1.apply(weights_init)

for ep in range(1,n_epochs_local):
    perm = np.random.permutation(n) # We use the "random reshuffling" version of SGD
    batches_data = np.array_split(xhat_0_cl1[perm,], n/bs)
    batches_mask = np.array_split(mask_cl1[perm,], n/bs)
    for it in range(len(batches_data)):
        optimizer_cl1.zero_grad()
        encoder_cl1.zero_grad()
        decoder_cl1.zero_grad()
        b_data = torch.from_numpy(batches_data[it]).float()
        b_mask = torch.from_numpy(batches_mask[it]).float()
        loss = miwae_loss(encoder = encoder_cl1,decoder = decoder_cl1, iota_x = b_data,mask = b_mask, d = d, p = p, K = K, batch_size = bs)
        loss.backward()
        optimizer_cl1.step()
    if ep % rounds == 1:
        print('Epoch %g' %ep)
        print('MIWAE likelihood bound  %g' %(-np.log(K)-miwae_loss(encoder = encoder_cl1,decoder = decoder_cl1, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(), d = d, p = p, K = K, batch_size = bs).cpu().data.numpy())) # Gradient step      
        print('Loss: {:.6f}'.format(loss.item()))

And we do the imputation on the same dataset:

In [ ]:
xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_cl1, decoder = decoder_cl1, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_local_cl1_data = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of local model on data from same client (cl 1)  %g' %err_local_cl1_data)
print('-----')

In [ ]:
n = data_test.shape[0] # number of observations
p = data_test.shape[1] # number of features
xhat_test = np.copy(xhat_test_local_std)
xhat_0_test = np.copy(xhat_0_test_local_std)
xfull_test = np.copy(xfull_test_local_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_cl1,decoder = decoder_cl1,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_local_cl1_test_data = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of local model on testing data %g' %err_local_cl1_test_data)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'Local_cl1',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[0])], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_cl1_test_data)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'Local_cl1', 
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[0])], 
            'N_rounds': 1, 'N_epochs': n_epochs*rounds,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_cl1_data)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

## 4. Local training on client 2,  and testing

In [ ]:
# Data

n = Clients_data[1].shape[0] # number of observations
p = Clients_data[1].shape[1] # number of features

xhat_cl2 = np.copy(xhat_cl2_local_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_local_std)
xfull_cl2 = np.copy(xfull_cl2_local_std)

encoder_cl2 = nn.Sequential(
    torch.nn.Linear(p, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 2*d),  # the encoder will output both the mean and the diagonal covariance
)

decoder_cl2 = nn.Sequential(
    torch.nn.Linear(d, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 3*p),  # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
)

optimizer_cl2 = torch.optim.Adam(list(encoder_cl2.parameters()) + list(decoder_cl2.parameters()),lr=1e-3)

def weights_init(layer):
    if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)
        
encoder_cl2.apply(weights_init)
decoder_cl2.apply(weights_init)

for ep in range(1,n_epochs_local):
    perm = np.random.permutation(n) # We use the "random reshuffling" version of SGD
    batches_data = np.array_split(xhat_0_cl2[perm,], n/bs)
    batches_mask = np.array_split(mask_cl2[perm,], n/bs)
    for it in range(len(batches_data)):
        optimizer_cl2.zero_grad()
        encoder_cl2.zero_grad()
        decoder_cl2.zero_grad()
        b_data = torch.from_numpy(batches_data[it]).float()
        b_mask = torch.from_numpy(batches_mask[it]).float()
        loss = miwae_loss(encoder = encoder_cl2,decoder = decoder_cl2, iota_x = b_data,mask = b_mask, d = d, p = p, K = K, batch_size = bs)
        loss.backward()
        optimizer_cl2.step()
    if ep % rounds == 1:
        print('Epoch %g' %ep)
        print('MIWAE likelihood bound  %g' %(-np.log(K)-miwae_loss(encoder = encoder_cl2,decoder = decoder_cl2, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(), d = d, p = p, K = K, batch_size = bs).cpu().data.numpy())) # Gradient step      
        print('Loss: {:.6f}'.format(loss.item()))

In [ ]:
xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder_cl2, decoder = decoder_cl2, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_local_cl2_data = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of local model on data from same client (cl 2)  %g' %err_local_cl2_data)
print('-----')

In [ ]:
n = data_test.shape[0] # number of observations
p = data_test.shape[1] # number of features
xhat_test = np.copy(xhat_test_local_std)
xhat_0_test = np.copy(xhat_0_test_local_std)
xfull_test = np.copy(xfull_test_local_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_cl2,decoder = decoder_cl2,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_local_cl2_test_data = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of local model on testing data %g' %err_local_cl2_test_data)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'Local_cl2',  
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[0])], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_cl2_test_data)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'Local_cl2', 
            'N_train_centers': N_cl, 'Size': [len(Clients_missing[0])], 
            'N_rounds': 1, 'N_epochs': n_epochs*rounds,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_cl2_data)}

with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    output_file.close()

## 6. Local training on centralized data:

We centralized data from all clients, and perform the local training, hence test results on the external testing dataset as well as data from client 1, as we deed for the others models.

In [ ]:
# Epochs

n_epochs_local = n_epochs*rounds*len(Clients_missing)

# Data

xmiss_tot = np.concatenate(Clients_missing,axis=0)

n = xmiss_tot.shape[0] # number of observations
p = xmiss_tot.shape[1] # number of features

mean_tot_missing = np.nanmean(xmiss_tot,0)
std_tot_missing = np.nanstd(xmiss_tot,0)
xmiss_tot = (xmiss_tot - mean_tot_missing)/std_tot_missing
mask_tot = np.isfinite(xmiss_tot) # binary mask that indicates which values are missing
xhat_0_tot = np.copy(xmiss_tot)
xhat_0_tot[np.isnan(xmiss_tot)] = 0
xhat_tot = np.copy(xhat_0_tot) # This will be out imputed data matrix

xfull_tot = np.concatenate(Clients_data,axis=0)
xfull_tot = (xfull_tot - mean_tot_missing)/std_tot_missing

# Model

encoder_tot = nn.Sequential(
    torch.nn.Linear(p, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 2*d),  # the encoder will output both the mean and the diagonal covariance
)

decoder_tot = nn.Sequential(
    torch.nn.Linear(d, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 3*p),  # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
)

optimizer_tot = torch.optim.Adam(list(encoder_tot.parameters()) + list(decoder_tot.parameters()),lr=1e-3)

def weights_init(layer):
    if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)
        
encoder_tot.apply(weights_init)
decoder_tot.apply(weights_init)

# Training loop

for ep in range(1,n_epochs_local):
    perm = np.random.permutation(n) # We use the "random reshuffling" version of SGD
    batches_data = np.array_split(xhat_0_tot[perm,], n/bs)
    batches_mask = np.array_split(mask_tot[perm,], n/bs)
    for it in range(len(batches_data)):
        optimizer_tot.zero_grad()
        encoder_tot.zero_grad()
        decoder_tot.zero_grad()
        b_data = torch.from_numpy(batches_data[it]).float()
        b_mask = torch.from_numpy(batches_mask[it]).float()
        loss = miwae_loss(encoder = encoder_tot,decoder = decoder_tot, iota_x = b_data,mask = b_mask, d = d, p = p, K = K, batch_size = bs)
        loss.backward()
        optimizer_tot.step()
    if ep % rounds == 1:
        print('Epoch %g' %ep)
        print('MIWAE likelihood bound  %g' %(-np.log(K)-miwae_loss(encoder = encoder_tot,decoder = decoder_tot, iota_x = torch.from_numpy(xhat_0_tot).float(),mask = torch.from_numpy(mask_tot).float(), d = d, p = p, K = K, batch_size = bs).cpu().data.numpy())) # Gradient step      

In [ ]:
n = Clients_data[0].shape[0] # number of observations
p = Clients_data[0].shape[1] # number of features

xhat_cl1 = np.copy(xhat_cl1_global_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_global_std)
xfull_cl1 = np.copy(xfull_cl1_global_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_tot, decoder = decoder_tot, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_local_tot_cl1_data_global_std = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of local model on the whole dataset, on data from same client 1  %g' %err_local_tot_cl1_data_global_std)
print('-----')

xhat_cl1 = np.copy(xhat_cl1_local_std)
xhat_0_cl1 = np.copy(xhat_0_cl1_local_std)
xfull_cl1 = np.copy(xfull_cl1_local_std)

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_tot, decoder = decoder_tot, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_local_tot_cl1_data_local_std = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of local model on the whole dataset, on data from same client 1 (with local standardization in client 1 data)  %g' %err_local_tot_cl1_data_local_std)
print('-----')

n = Clients_data[1].shape[0] # number of observations
p = Clients_data[1].shape[1] # number of features

xhat_cl2 = np.copy(xhat_cl2_global_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_global_std)
xfull_cl2 = np.copy(xfull_cl2_global_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder_tot, decoder = decoder_tot, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_local_tot_cl2_data_global_std = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of local model on the whole dataset, on data from same client 2  %g' %err_local_tot_cl2_data_global_std)
print('-----')

xhat_cl2 = np.copy(xhat_cl2_local_std)
xhat_0_cl2 = np.copy(xhat_0_cl2_local_std)
xfull_cl2 = np.copy(xfull_cl2_local_std)

xhat_cl2[~mask_cl2] = miwae_impute(encoder = encoder_tot, decoder = decoder_tot, iota_x = torch.from_numpy(xhat_0_cl2).float(),mask = torch.from_numpy(mask_cl2).float(),d = d,L= L).cpu().data.numpy()[~mask_cl2]
err_local_tot_cl2_data_local_std = np.array([mse(xhat_cl2,xfull_cl2,mask_cl2)])
print('Imputation MSE of local model on the whole dataset, on data from same client 2 (with local standardization in client 2 data)  %g' %err_local_tot_cl2_data_local_std)
print('-----')

n = data_test.shape[0] # number of observations
p = data_test.shape[1] # number of features

xhat_test = np.copy(xhat_test_global_std)
xhat_0_test = np.copy(xhat_0_test_global_std)
xfull_test = np.copy(xfull_test_global_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_tot,decoder = decoder_tot,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_local_tot_test_data_global_std = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of local model on the whole dataset, on testing data %g' %err_local_tot_test_data_global_std)
print('-----')

xhat_test = np.copy(xhat_test_local_std)
xhat_0_test = np.copy(xhat_0_test_local_std)
xfull_test = np.copy(xfull_test_local_std)

xhat_test[~mask_test] = miwae_impute(encoder = encoder_tot,decoder = decoder_tot,iota_x = torch.from_numpy(xhat_0_test).float(),mask = torch.from_numpy(mask_test).float(),d = d,L= L).cpu().data.numpy()[~mask_test]
err_local_tot_test_data_local_std = np.array([mse(xhat_test,xfull_test,mask_test)])
print('Imputation MSE of local model on the whole dataset, on testing data (with local standardization in testing data) %g' %err_local_tot_test_data_local_std)
print('-----')

In [ ]:
# list of column names
field_names = ['Split_type', 'Test_data', 'model', 'N_train_centers', 'Size', 
               'N_rounds', 'N_epochs', 'std_training', 'std_testing', 'MSE']

# Dictionary
dict_out_1={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'Centralized', 
            'N_train_centers': N_cl, 'Size': [len(xmiss_tot)], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_tot_test_data_local_std)}
dict_out_2={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'Centralized', 
            'N_train_centers': N_cl, 'Size': [len(xmiss_tot)], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_tot_cl1_data_local_std)}
dict_out_3={'Split_type': Split_type, 'Test_data': 'Test', 'model': 'Centralized', 
            'N_train_centers': N_cl, 'Size': [len(xmiss_tot)], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'global', 'MSE': float(err_local_tot_test_data_global_std)}
dict_out_4={'Split_type': Split_type, 'Test_data': 'Client1', 'model': 'Centralized', 
            'N_train_centers': N_cl, 'Size': [len(xmiss_tot)], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'global', 'MSE': float(err_local_tot_cl1_data_global_std)}
dict_out_5={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'Centralized', 
            'N_train_centers': N_cl, 'Size': [len(xmiss_tot)], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'local', 'MSE': float(err_local_tot_cl2_data_local_std)}

dict_out_6={'Split_type': Split_type, 'Test_data': 'Client2', 'model': 'Centralized', 
            'N_train_centers': N_cl, 'Size': [len(xmiss_tot)], 
            'N_rounds': 1, 'N_epochs': n_epochs_local,
          'std_training': 'Loc', 'std_testing': 'global', 'MSE': float(err_local_tot_cl2_data_global_std)}



with open('results/output_notiid.csv', 'a') as output_file:
    dictwriter_object = csv.DictWriter(output_file, fieldnames=field_names)
    dictwriter_object.writerow(dict_out_1)
    dictwriter_object.writerow(dict_out_2)
    dictwriter_object.writerow(dict_out_3)
    dictwriter_object.writerow(dict_out_4)
    dictwriter_object.writerow(dict_out_5)
    dictwriter_object.writerow(dict_out_6)
    output_file.close()

## 7. Summary of obtained results:

In [ ]:
from tabulate import tabulate

print('Imputation MSE on testing data')
print('-----')
data = [['FedAvg, global std', err_test_data_global_std],
['FedAvg, local std', err_test_data_local_std],
['FedProx, global std', err_test_data_fedprox_global_std],
['FedProx, local std', err_test_data_fedprox_local_std], 
['FedLocStd, local std', err_test_data_fedprox_std_local],  
['Local (cl1), local std', err_local_cl1_test_data], 
['Local (cl2), local std', err_local_cl2_test_data],  
['Centralized, global std', err_local_tot_test_data_global_std],
['Centralized, local std', err_local_tot_test_data_local_std]]
print (tabulate(data, headers=["Model", "Mean Squared Error (\u2193)"]))
print('-----')
print('-----')
print('Imputation MSE on local data from client 1')
print('-----')
data = [['FedAvg, global std', err_cl1_data_global_std], 
['FedAvg, local std', err_cl1_data_local_std],  
['FedProx, global std', err_cl1_data_fedprox_global_std],
['FedProx, local std', err_cl1_data_fedprox_local_std],
['FedLocStd, local std', err_cl1_data_fedprox_std_local], 
['Local (cl1), local std', err_local_cl1_data], 
['Centralized, global std', err_local_tot_cl1_data_global_std],
['Centralized, local std', err_local_tot_cl1_data_local_std]]
print (tabulate(data, headers=["Model", "Mean Squared Error (\u2193)"]))
print('-----')
print('-----')
print('Imputation MSE on local data from client 2')
print('-----')
data = [['FedAvg, global std', err_cl2_data_global_std], 
['FedAvg, local std', err_cl2_data_local_std], 
['FedProx, global std', err_cl2_data_fedprox_global_std],
['FedProx, local std', err_cl2_data_fedprox_local_std],
['FedLocStd, local std', err_cl2_data_fedprox_std_local], 
['Local (cl2), local std', err_local_cl2_data],
['Centralized, global std', err_local_tot_cl2_data_global_std],
['Centralized, local std', err_local_tot_cl2_data_local_std]]
print (tabulate(data, headers=["Model", "Mean Squared Error (\u2193)"]))